In [1]:
import numpy as np
from models import dist_model as dm
from data import data_loader as dl
import argparse
from IPython import embed
import torch
import torch.nn as nn
from collections import defaultdict
from data.dataset.twoafc_dataset import TwoAFCDataset
import matplotlib.pyplot as plt
import pickle
import os

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
is_cuda = torch.cuda.is_available()

datasets = ['val/traditional','val/cnn','val/superres','val/deblur','val/color','val/frameinterp']
dataset_mode = '2afc' # 2afc or jnd

models = []

# Grayscale models

In [ ]:
# traditional Baselines

m = dm.DistModel()
m.initialize(model='L1',colorspace='Gray',use_gpu=is_cuda)
m.model_name = 'L1 [LA]'
models.append(m)

m = dm.DistModel()
m.initialize(model='L2',colorspace='Gray',use_gpu=is_cuda)
m.model_name = 'L2 [LA]'
models.append(m)

m = dm.DistModel()
m.initialize(model='ssim',colorspace='Gray',use_gpu=is_cuda)
m.model_name = 'SSIM [LA]'
models.append(m)

In [ ]:
# watson models

m = dm.DistModel()
path=os.path.join('./checkpoints/', 'gray_watson_dct_trial0', 'latest_net_.pth')
m.initialize(model='watson', net='dct', colorspace='Gray',use_gpu=is_cuda)
m.net.load_state_dict(torch.load(path))
m.model_name = 'Watson-dct [LA]'
models.append(m)

m = dm.DistModel()
path=os.path.join('./checkpoints/', 'gray_watson_fft_trial0', 'latest_net_.pth')
m.initialize(model='watson', net='fft', colorspace='Gray',use_gpu=is_cuda)
m.net.load_state_dict(torch.load(path))
m.model_name = 'Watson-fft [LA]'
models.append(m)

In [ ]:
# watson vgg
m = dm.DistModel()
path=os.path.join('./checkpoints/', 'gray_watson_vgg_trial0', 'latest_net_.pth')
m.initialize(model='watson', net='vgg', colorspace='Gray',use_gpu=is_cuda)
m.net.load_state_dict(torch.load(path))
m.model_name = 'Watson-vgg [LA]'
m.batch_size = m.batch_size // 4
models.append(m)

In [ ]:
# Deeploss competitor: not tuned vgg, all 5 layers even weight
m = dm.DistModel()
path=os.path.join('./checkpoints/', 'gray_pnet_lin_vgg_trial0', 'latest_net_.pth')
m.initialize(model='net-lin', net='vgg', model_path=path, colorspace='Gray',use_gpu=is_cuda, batch_size=64)
m.model_name = 'Deeploss-vgg [LA]'
m.batch_size = m.batch_size // 4
models.append(m)

# color models

In [ ]:
# traditional Baselines
m = dm.DistModel()
m.initialize(model='L1',colorspace='RGB',use_gpu=is_cuda)
m.model_name = 'L1'
models.append(m)

m = dm.DistModel()
m.initialize(model='L2',colorspace='RGB',use_gpu=is_cuda)
m.model_name = 'L2'
models.append(m)

m = dm.DistModel()
m.initialize(model='ssim',colorspace='RGB',use_gpu=is_cuda)
m.model_name = 'SSIM'
models.append(m)

In [ ]:
# watson models

m = dm.DistModel()
path=os.path.join('./checkpoints/', 'rgb_watson_dct_trial0', 'latest_net_.pth')
m.initialize(model='watson', net='dct', colorspace='RGB',use_gpu=is_cuda)
m.net.load_state_dict(torch.load(path))
m.model_name = 'Watson-dct'
models.append(m)

m = dm.DistModel()
path=os.path.join('./checkpoints/', 'rgb_watson_fft_trial0', 'latest_net_.pth')
m.initialize(model='watson', net='fft', colorspace='RGB',use_gpu=is_cuda)
m.net.load_state_dict(torch.load(path))
m.model_name = 'Watson-fft'
models.append(m)

In [ ]:
# watson vgg
m = dm.DistModel()
path=os.path.join('./checkpoints/', 'rgb_watson_vgg_trial0', 'latest_net_.pth')
m.initialize(model='watson', net='vgg', colorspace='RGB',use_gpu=is_cuda)
m.net.load_state_dict(torch.load(path))
m.model_name = 'Watson-vgg'
m.batch_size = m.batch_size // 4
models.append(m)

In [ ]:
# Deeploss competitor: not tuned vgg, all 5 layers even weight
m = dm.DistModel()
path=os.path.join('./checkpoints/', 'rgb_pnet_lin_vgg_trial0', 'latest_net_.pth')
m.initialize(model='net-lin', net='vgg', model_path=path, colorspace='RGB',use_gpu=is_cuda, batch_size=64)
m.model_name = 'Deeploss-vgg'
m.batch_size = m.batch_size // 4
models.append(m)

In [ ]:
def ddict(d):
    return defaultdict(lambda: defaultdict(lambda: {}), d)

def ddict2dict(d):
    for k, v in d.items():
        if isinstance(v, dict):
            d[k] = ddict2dict(v)
    return dict(d)

def dict2ddict(d):
    for k, v in d.items():
        if isinstance(v, dict):
            d[k] = dict2ddict(v)
    return ddict(d)

In [5]:
# initialize data loader
def eval_loss_metric(model, resultdict=None):
    if resultdict is None:
        resultdict = defaultdict(lambda: defaultdict(lambda: {}))
    
    for dataset in datasets:
        
        data_loader = dl.CreateDataLoader(dataset, dataset_mode=dataset_mode, batch_size=model.batch_size)

        # evaluate model on data
        if(dataset_mode=='2afc'):
            (score, results_verbose) = dm.score_2afc_dataset(data_loader, model.forward)
            resultdict[model.model_name][dataset]['score'] = results_verbose['scores'].mean()
            resultdict[model.model_name][dataset]['std'] = results_verbose['scores'].std()
            
            human_judgements = results_verbose['gts']
            human_scores = human_judgements**2 + (1 - human_judgements)**2
            resultdict['Human'][dataset]['score'] = human_scores.mean()
            resultdict['Human'][dataset]['std'] = human_scores.std()
            
            
        elif(dataset_mode=='jnd'):
            raise Exception('not implemented / validated')
            (score, results_verbose) = dm.score_jnd_dataset(data_loader, model.forward)


        # print results
        print(' Model [%s]  Dataset [%s]: %.2f'%(model.model_name, dataset, 100.*score))
    return resultdict


In [ ]:
res = dict2ddict(pickle.load(open("eval_results.p", "rb")))
#res = defaultdict(lambda: defaultdict(lambda: {}))

for i, model in enumerate(models):
    print('eval model {} / {}: "{}"'.format(i+1, len(models), model.model_name))
    eval_loss_metric(model, res)

# save results
pickle.dump(ddict2dict(res), open("eval_results.p", "wb"))

# FFT sensitivity vs phase weights

In [3]:
import sys
sys.path.append('../loss')
from watson_fft_evaluating import WatsonDistanceFftEvaluating
from tqdm import tqdm_notebook as tqdm

datasets = ['val/traditional','val/cnn'] #,'val/superres','val/deblur','val/color','val/frameinterp']

def eval_fft_amp_vs_phase_weighting(amp_factor, phase_factor):
    # set up model
    m = dm.DistModel()
    path=os.path.join('./checkpoints/', 'gray_watson_fft_trial0', 'latest_net_.pth')
    m.initialize(model='watson', net='fft', colorspace='Gray',use_gpu=is_cuda)
    m.net.net = WatsonDistanceFftEvaluating(reduction = 'none')
    m.net.load_state_dict(torch.load(path))
    m.net.net.a = amp_factor
    m.net.net.b = phase_factor
    m.net = m.net.to(device)
    
    # eval
    resultdict = eval_loss_metric(m)
    score = sum([resultdict[m.model_name][d]['score'] for d in datasets]) / len(datasets)
    error = sum([resultdict[m.model_name][d]['std'] for d in datasets]) / len(datasets)
    
    # get balance
    amps = m.net.net.total_amplitude_term_dist
    phase = m.net.net.total_phase_term_dist
    
    pct_amp = amps / (amps + phase)
    pct_phase = phase / (amps + phase)
    
    return pct_amp, pct_phase, score, error

In [6]:
alphas = [0] + list(np.logspace(-3, 0, num=100))
res = {'frac_amps' : [], 'alpha':[], 'scores' : [], 'errors' : []}
for a in tqdm(alphas):
    pct_amp, pct_phase, score, error = eval_fft_amp_vs_phase_weighting(a, 1 - a)
    res['frac_amps'].append(pct_amp)
    res['alpha'].append(a)
    res['scores'].append(score)
    res['errors'].append(error)
    
pickle.dump(res, open("fft_amp_phase_weighting.p", "wb"))

/home/steffen/study/VAE-perceptual-loss/venv-pytorch/lib/python3.5/site-packages/torchvision/transforms/transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


 Model [Watson]  Dataset [val/traditional]: 55.02
 Model [Watson]  Dataset [val/cnn]: 75.35
 Model [Watson]  Dataset [val/traditional]: 56.78
 Model [Watson]  Dataset [val/cnn]: 76.19
 Model [Watson]  Dataset [val/traditional]: 56.88
 Model [Watson]  Dataset [val/cnn]: 76.23
 Model [Watson]  Dataset [val/traditional]: 56.87
 Model [Watson]  Dataset [val/cnn]: 76.26
 Model [Watson]  Dataset [val/traditional]: 57.02
 Model [Watson]  Dataset [val/cnn]: 76.35
 Model [Watson]  Dataset [val/traditional]: 57.17
 Model [Watson]  Dataset [val/cnn]: 76.44
 Model [Watson]  Dataset [val/traditional]: 57.28
 Model [Watson]  Dataset [val/cnn]: 76.52
 Model [Watson]  Dataset [val/traditional]: 57.40
 Model [Watson]  Dataset [val/cnn]: 76.57
 Model [Watson]  Dataset [val/traditional]: 57.48
 Model [Watson]  Dataset [val/cnn]: 76.56
 Model [Watson]  Dataset [val/traditional]: 57.57
 Model [Watson]  Dataset [val/cnn]: 76.60
 Model [Watson]  Dataset [val/traditional]: 57.67
 Model [Watson]  Dataset [val/

 Model [Watson]  Dataset [val/cnn]: 78.01
 Model [Watson]  Dataset [val/traditional]: 63.63
 Model [Watson]  Dataset [val/cnn]: 78.03
 Model [Watson]  Dataset [val/traditional]: 63.63
 Model [Watson]  Dataset [val/cnn]: 78.05
 Model [Watson]  Dataset [val/traditional]: 63.67
 Model [Watson]  Dataset [val/cnn]: 78.05
 Model [Watson]  Dataset [val/traditional]: 63.66
 Model [Watson]  Dataset [val/cnn]: 78.07
 Model [Watson]  Dataset [val/traditional]: 63.66
 Model [Watson]  Dataset [val/cnn]: 78.07
 Model [Watson]  Dataset [val/traditional]: 63.66
 Model [Watson]  Dataset [val/cnn]: 78.07
 Model [Watson]  Dataset [val/traditional]: 63.67
 Model [Watson]  Dataset [val/cnn]: 78.07
 Model [Watson]  Dataset [val/traditional]: 63.67
 Model [Watson]  Dataset [val/cnn]: 78.07
 Model [Watson]  Dataset [val/traditional]: 63.67
 Model [Watson]  Dataset [val/cnn]: 78.07
 Model [Watson]  Dataset [val/traditional]: 63.66
 Model [Watson]  Dataset [val/cnn]: 78.06
 Model [Watson]  Dataset [val/traditio